In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_parquet('taxi_data.parquet')
df_train.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-06-01 00:08:48,2023-06-01 00:29:41,1.0,3.40,1.0,N,140,238,1,21.9,3.50,0.5,6.70,0.0,1.0,33.60,2.5,0.00
1,1,2023-06-01 00:15:04,2023-06-01 00:25:18,0.0,3.40,1.0,N,50,151,1,15.6,3.50,0.5,3.00,0.0,1.0,23.60,2.5,0.00
2,1,2023-06-01 00:48:24,2023-06-01 01:07:07,1.0,10.20,1.0,N,138,97,1,40.8,7.75,0.5,10.00,0.0,1.0,60.05,0.0,1.75
3,2,2023-06-01 00:54:03,2023-06-01 01:17:29,3.0,9.83,1.0,N,100,244,1,39.4,1.00,0.5,8.88,0.0,1.0,53.28,2.5,0.00
4,2,2023-06-01 00:18:44,2023-06-01 00:27:18,1.0,1.17,1.0,N,137,234,1,9.3,1.00,0.5,0.72,0.0,1.0,15.02,2.5,0.00


In [ ]:
df_train.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

In [ ]:
df_train = df_train.loc[df_train['RatecodeID'] <= 6]

In [ ]:
nan_df = df_train[df_train.isna().any(axis = 1)]
df_drop = df_train.dropna(subset = ['passenger_count'])
nan_df = df_drop[df_drop.isna().any(axis = 1)]
df_train = df_drop
df_train= df_train[df_train['fare_amount'] > 0]
df_train = df_train[df_train['fare_amount'] < 350000]
df_train = df_train[df_train['trip_distance'] < 350]
df_train = df_train[df_train['trip_distance'] > 0]
df_train = df_train[df_train['passenger_count'] < 7]

In [ ]:
df_train = df_train.drop_duplicates().reset_index(drop=True)
df_train['trip_id'] = df_train.index

In [ ]:
df_train['tpep_pickup_datetime'] = pd.to_datetime(df_train['tpep_pickup_datetime'])
df_train['tpep_dropoff_datetime'] = pd.to_datetime(df_train['tpep_dropoff_datetime'])

In [ ]:
datetime_dim = df_train[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop = True)
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday
datetime_dim['drop_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday
datetime_dim['datetime_id'] = datetime_dim.index

datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]

In [ ]:
datetime_dim.to_parquet('datetime_dim.parquet', index = False)


In [ ]:
passenger_count_dim = df_train[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]


In [ ]:
passenger_count_dim.to_parquet('passenger_count_dim.parquet', index = False)

In [ ]:
trip_distance_dim = df_train[['trip_distance']].reset_index(drop = True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]

In [ ]:
trip_distance_dim.to_parquet('trip_distance_dim.parquet', index = False)

In [ ]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}
rate_code_dim = df_train[['RatecodeID']].reset_index(drop = True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id', 'RatecodeID','rate_code_name']]

In [ ]:
rate_code_dim.to_parquet('rate_code_dim.parquet',index = False)


In [ ]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = df_train[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]

In [ ]:
payment_type_dim.to_parquet('payment_dype_dim.parquet',index = False)

In [ ]:
pickup_location_dim = df_train[['PULocationID']].reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[['pickup_location_id','PULocationID']]

dropoff_location_dim = df_train[['DOLocationID']].reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id','DOLocationID']]

In [ ]:
pickup_location_dim.to_parquet('pickup_location_dim.parquet',index = False)

In [ ]:
dropoff_location_dim.to_parquet('dropoff_location_dim.parquet',index = False)

In [ ]:
extra_fee_dim = df_train[['extra','mta_tax','tolls_amount', 'improvement_surcharge','congestion_surcharge', 'Airport_fee']].reset_index(drop=True)
extra_fee_dim['extra_fee_id'] = extra_fee_dim.index

extra_fee_dim = extra_fee_dim[['extra_fee_id', 'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'Airport_fee']]

In [ ]:
extra_fee_dim.to_parquet('extra_fee_dim.parquet',index = False)

In [ ]:
fact_table = df_train.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             .merge(extra_fee_dim, left_on='trip_id', right_on='extra_fee_id')\
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'total_amount']]

In [ ]:
fact_table.to_parquet('fact_table.parquet',index = False)